In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from keras.optimizers import RMSprop
from keras.utils import to_categorical
from keras_preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

from keras.models import Sequential
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)


Num GPUs Available:  1


In [2]:
train_data = pd.read_csv('./data/train.csv')
test_data = pd.read_csv('./data/test.csv')

In [3]:
train_y_d = train_data['label']
train_x_d = train_data.drop(['label'], axis=1)

In [4]:
train_y_d.value_counts()

label
1    4684
7    4401
3    4351
9    4188
2    4177
6    4137
0    4132
4    4072
8    4063
5    3795
Name: count, dtype: int64

In [5]:
train_x_d.isnull().sum()


pixel0      0
pixel1      0
pixel2      0
pixel3      0
pixel4      0
           ..
pixel779    0
pixel780    0
pixel781    0
pixel782    0
pixel783    0
Length: 784, dtype: int64

In [6]:
test_data.isnull().sum()

pixel0      0
pixel1      0
pixel2      0
pixel3      0
pixel4      0
           ..
pixel779    0
pixel780    0
pixel781    0
pixel782    0
pixel783    0
Length: 784, dtype: int64

In [7]:
train_x_d.shape


(42000, 784)

In [8]:
test_data.shape

(28000, 784)

In [9]:
train_x_d /= 255.0

In [10]:
test_data /= 255.0

In [11]:
train_x_d = train_x_d.values.reshape(-1, 28, 28, 1)
test_data = test_data.values.reshape(-1, 28, 28, 1)
y_train_d = to_categorical(train_y_d, 10)

In [12]:
train_x, val_x, train_y, val_y = train_test_split(train_x_d, y_train_d, test_size=0.1, random_state=42)

train_x.shape

(37800, 28, 28, 1)

In [13]:
augmented_data = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2
)
augmented_data.fit(train_x)

In [14]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), activation='relu',padding='same',input_shape=(28, 28, 1)))
model.add(Conv2D(32, kernel_size=(5, 5), activation='relu',padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu',padding='same'))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu',padding='same'))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu',padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))


optimiser = RMSprop(learning_rate=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
model.compile(loss='categorical_crossentropy',optimizer=optimiser,metrics=['accuracy'])

learning_rate_reduction_ann = ReduceLROnPlateau(
    monitor='val_accuracy',
    patience=5,
    verbose=1,
    factor=0.5,
    min_lr=0.00001,
)
batch_size = 64
epochs = 10

/opt/anaconda3/envs/KS-DR/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-08-31 01:46:21.640625: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-08-31 01:46:21.641099: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-08-31 01:46:21.641110: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-08-31 01:46:21.641833: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-08-31 01:46:21.641900: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:

In [15]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [16]:
evaluations = model.fit(
    augmented_data.flow(train_x, train_y, batch_size=batch_size),
    steps_per_epoch=len(train_x) // batch_size,
    epochs=epochs,
    validation_data=(val_x,val_y),
    callbacks=[learning_rate_reduction_ann],
    verbose=1
)

Epoch 1/10


2024-08-31 01:46:27.561770: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
/opt/anaconda3/envs/KS-DR/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


590/590 ━━━━━━━━━━━━━━━━━━━━ 22s 34ms/step - accuracy: 0.5302 - loss: 1.3495 - val_accuracy: 0.9726 - val_loss: 0.0912 - learning_rate: 0.0010
Epoch 2/10
  1/590 ━━━━━━━━━━━━━━━━━━━━ 48s 82ms/step - accuracy: 0.9219 - loss: 0.3235

2024-08-31 01:46:49.342015: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-08-31 01:46:49.342048: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[StatefulPartitionedCall/ArgMax/_32]]
2024-08-31 01:46:49.342064: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 3475367637929681113
2024-08-31 01:46:49.342080: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 9336517979975189235
2024-08-31 01:46:49.342083: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 7681335450985080455
2024-08-31 01:46:49.342093: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 183059407985291

590/590 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9219 - loss: 0.3235 - val_accuracy: 0.9681 - val_loss: 0.1074 - learning_rate: 0.0010
Epoch 3/10
590/590 ━━━━━━━━━━━━━━━━━━━━ 20s 34ms/step - accuracy: 0.9362 - loss: 0.2288 - val_accuracy: 0.9805 - val_loss: 0.0667 - learning_rate: 0.0010
Epoch 4/10
  1/590 ━━━━━━━━━━━━━━━━━━━━ 26s 46ms/step - accuracy: 0.9688 - loss: 0.0839

...

590/590 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 1.0000 - loss: 0.0470 - val_accuracy: 0.9883 - val_loss: 0.0416 - learning_rate: 0.0010
Epoch 7/10
590/590 ━━━━━━━━━━━━━━━━━━━━ 21s 35ms/step - accuracy: 0.9707 - loss: 0.1112 - val_accuracy: 0.9850 - val_loss: 0.0424 - learning_rate: 0.0010
Epoch 8/10
  1/590 ━━━━━━━━━━━━━━━━━━━━ 29s 49ms/step - accuracy: 0.9688 - loss: 0.0634

2024-08-31 01:47:53.690162: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-08-31 01:47:53.690194: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 6066944132696594584
2024-08-31 01:47:53.690197: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_19]]
2024-08-31 01:47:53.690207: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 6900713536446656802
2024-08-31 01:47:53.690210: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 271373859302420507
2024-08-31 01:47:53.690214: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 13718429492427969067
2024-08-31

590/590 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9688 - loss: 0.0634 - val_accuracy: 0.9838 - val_loss: 0.0522 - learning_rate: 0.0010
Epoch 9/10
590/590 ━━━━━━━━━━━━━━━━━━━━ 24s 40ms/step - accuracy: 0.9745 - loss: 0.0921 - val_accuracy: 0.9829 - val_loss: 0.0583 - learning_rate: 0.0010
Epoch 10/10
  1/590 ━━━━━━━━━━━━━━━━━━━━ 5:23 549ms/step - accuracy: 0.9844 - loss: 0.0877

2024-08-31 01:48:19.561994: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-08-31 01:48:19.562247: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[StatefulPartitionedCall/ArgMax/_32]]
2024-08-31 01:48:19.562498: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 3475367637929681113
2024-08-31 01:48:19.562626: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 9336517979975189235
2024-08-31 01:48:19.562632: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 7681335450985080455
2024-08-31 01:48:19.562636: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 183059407985291


Epoch 10: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
590/590 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.9844 - loss: 0.0877 - val_accuracy: 0.9871 - val_loss: 0.0469 - learning_rate: 0.0010


In [18]:
results = model.predict(test_data)

# select the indix with the maximum probability
results = np.argmax(results,axis = 1)

results = pd.Series(results,name="Label")
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

submission.to_csv("submission_first.csv",index=False)

875/875 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step
